In [ ]:
from league import RiotAPI, League
from config import Config
import logging
import time
import json
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
secrets = Config()
api_key = secrets.get("api_key")


In [ ]:
config = Config()
api = RiotAPI(config)
league = League("totis", api)

In [ ]:
last_match = league.get_matches(count=1)

In [ ]:
last_match

In [ ]:
last_match_info = league.get_match_info(last_match[0])

In [ ]:
last_match_info

In [ ]:
main_keys = list(last_match_info["info"]["participants"][0].keys())
main_keys

In [ ]:
last_match_info["info"]["participants"][0]["challenges"].keys()

In [ ]:
challenges={
    "abilityUses": 186,
    "bountyGold": 1275,
    "controlWardsPlaced": 0,
    "damagePerMinute": 1777.0054566655733,
    "dodgeSkillShotsSmallWindow": 46,
    "firstTurretKilled": 1,
    "firstTurretKilledTime": 531.0225707,
    "goldPerMinute": 783.3466657043723,
    "initialBuffCount": 0,
    "initialCrabCount": 0,
    "jungleCsBefore10Minutes": 0,
    "landSkillShotsEarlyGame": 101,
    "laneMinionsFirst10Minutes": 26,
    "outerTurretExecutesBefore10Minutes": 0,
    "outnumberedKills": 3,
    "scuttleCrabKills": 0,
    "skillshotsDodged": 273,
    "skillshotsHit": 175,
    "snowballsHit": 0,
    "soloKills": 0,
    "stealthWardsPlaced": 0,
    "turretPlatesTaken": 0,
    "turretTakedowns": 1,
    "visionScorePerMinute": 0,
    "wardTakedowns": 0,
}
challenges_keys = challenges.keys()

In [ ]:
challenges_keys 

In [ ]:
matches = []

In [ ]:
# While no error run a loop
i=0
s=0
while s<15000:
    try:
        matches.append(league.get_matches(start=s,count=100))
    except Exception as e:
        logging.info(f"Error while getting matches in loop {i}: {e}")
        break
    i+=1
    s+=100
    time.sleep(1.5)

In [ ]:
len(matches)

In [ ]:
with open("matches.json", "w") as f:
    json.dump(matches, f)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(matches)

In [ ]:
df.to_csv("matches.csv")

In [ ]:
df

In [ ]:
#load json file
with open("./data/matches.json", "r") as f:
    matches = json.load(f)

In [ ]:
#make it into 1 single list with no empty values
matches = [match for sublist in matches for match in sublist if match]

In [ ]:
matches

In [ ]:
# Get match info
historic_match_info = []
for match in tqdm(matches):
    try:
        historic_match_info.append(league.get_match_info(match))
    except Exception as e:
        logging.info(f"Error while getting match info for match {match}: {e}")
        continue
    time.sleep(1.5)

In [ ]:
#store it in a json file
with open("historic_match_info.json", "w") as f:
    json.dump(historic_match_info, f)

In [ ]:
# In this loop we only focus on totis
challenge_keys_to_keep = list(challenges_keys)
participant_keys_to_keep = main_keys

historic_players = []
historic_matches_db = []

for match_info in historic_match_info:
    match_data = {
        "match_id": match_info["metadata"]["matchId"],
        "game_duration": match_info["info"]["gameDuration"],
        "game_mode": match_info["info"]["gameMode"],
        "game_start_time": match_info["info"]["gameStartTimestamp"]
    }

    for participant in match_info["info"]["participants"]:
        if participant["summonerName"] == "totis":
            participant_data = {key: participant[key] for key in participant_keys_to_keep if key in participant}
            match_data.update(participant_data)
        else:
            player_data = {
                "summoner_name": participant["summonerName"],
                "summoner_id": participant["summonerId"],
                "puuid": participant["puuid"],
                "match_id": match_info["metadata"]["matchId"]
            }
            historic_players.append(player_data)


    
    historic_matches_db.append(match_data)

In [ ]:
totis_df = pd.DataFrame(historic_matches_db)
totis_df.to_csv("historic_matches_db.csv")
players = pd.DataFrame(historic_players)
players.to_csv("historic_players.csv")


In [ ]:
#load
totis_df= pd.read_csv("./data/historic_matches_db.csv")
players = pd.read_csv("./data/historic_players.csv")

In [ ]:
#return this with minute resolution
totis_df["game_date"] = pd.to_datetime(totis_df["game_start_time"], unit="ms").dt.floor('S')
totis_df["game_date"].head()
# ren

In [ ]:
totis_df.to_csv("historic_matches_db.csv")

In [ ]:
list(totis_df.columns)

In [ ]:
players

In [ ]:
players["summoner_name"].value_counts().head(15).plot(kind="barh")
plt.title("Top 15 players with most games played")
plt.bar_label(plt.gca().containers[0])
plt.xlabel("Games played")
plt.show()

In [ ]:
totis_df["game_mode"].value_counts().plot(kind="barh")
plt.title("Game modes played")
plt.bar_label(plt.gca().containers[0])
plt.xlabel("Games played")
plt.show()

In [ ]:
totis_df["game_start_time"] = pd.to_datetime(totis_df["game_start_time"], unit="ms")

In [ ]:
totis_df = totis_df.sort_values("game_start_time")

In [ ]:
totis_df["kills"].plot(kind="hist")

In [ ]:
totis_df.groupby("championName")["kills"].mean().sort_values(ascending=False).head(15).plot(kind="barh")
#plot also total count of games played with that champion

plt.title("Top 15 champions with most kills")
plt.bar_label(plt.gca().containers[0])
plt.xlabel("Average kills")
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

totis_df["championName"].value_counts().head(15).sort_values(ascending=True).plot(kind="barh", ax=axes[0])
axes[0].set_title("Top 15 champions with most games played")
axes[0].set_xlabel("Games played")

totis_df["championName"].value_counts().tail(15).sort_values(ascending=True).plot(kind="barh", ax=axes[1])
axes[1].set_title("Top 15 champions with least games played")
axes[1].set_xlabel("Games played")

plt.tight_layout()
plt.show()


In [ ]:
totis_df["win"].value_counts().plot(kind="pie", autopct="%.2f%%")